In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Langchain Tutorial Prompts"

In [5]:
prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [6]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
    ])

In [7]:
prompt_template.input_variables

['context', 'query']

In [8]:
print(prompt_template.format(query="What is Langchain?", context="Langchain is a framework for building applications with LLMs."))

System: 
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: Langchain is a framework for building applications with LLMs.

Human: What is Langchain?


In [9]:
from pprint import pprint
pprint(prompt_template.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.\nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]


## Initializing the LLM

In [10]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="groq:openai/gpt-oss-20b", temperature=0)

In [14]:
# Creating a Chain Object
pipeline = (
    prompt_template 
    | llm 
    | (lambda x: x.content)
)

In [15]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [17]:
from IPython.display import display, Markdown
display(Markdown(pipeline.invoke({"context":context, "query":query})))

Aurelio AI builds tooling and services for AI engineers, with a focus on language‑AI. Their work includes:

- **Open‑source frameworks** such as Semantic Router and Semantic Chunkers that help engineers structure and route language model queries.
- **An AI Platform** that supplies developers with ready‑made components and workflows for building and deploying language‑AI applications.
- **Custom development services** that partner with other organizations to turn their AI ideas into production‑ready products.

In September 2024 they were recognized as LangChain Experts, reflecting their deep experience in creating solutions with the LangChain ecosystem.

## Few Shot Prompting

In [18]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

In [19]:
examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"}
]

In [21]:
from langchain.prompts import FewShotChatMessagePromptTemplate 

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt= example_prompt,
    examples = examples
)

print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


### Few-Shot Example

In [28]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

new_prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    ("user", "{query}"),
])

new_pipeline = (
    new_prompt_template
    | llm
    | (lambda x : x.content)
)

out = new_pipeline.invoke({"query": query, "context": context})
display(Markdown(out))

## What Aurelio AI Does

**Aurelio AI is a specialized AI engineering company that builds tools, frameworks, and services to help other engineers create, deploy, and scale language‑model‑based applications.**

---

### Key Offerings

- **Open‑source frameworks**
  - **Semantic Router** – directs user queries to the most relevant model or data source.
  - **Semantic Chunkers** – splits large documents into semantically meaningful chunks for efficient retrieval.

- **AI Platform**
  - A suite of developer tools that streamline the end‑to‑end workflow of building AI applications (model selection, data ingestion, evaluation, deployment).

- **Professional Services**
  - Consulting and custom development for organizations looking to integrate AI into their products or services.

- **LangChain Expertise**
  - Recognized as LangChain Experts (since September 2024) after extensive experience building solutions with the LangChain ecosystem.

---

### Core Focus Areas

- **Language AI** – building and optimizing large‑language‑model (LLM) applications.
- **AI Agents** – designing autonomous agents that can reason, plan, and act.
- **Information Retrieval** – leveraging semantic search and retrieval‑augmented generation to improve accuracy and relevance.

---

### How They Help

- Provide reusable, battle‑tested components that reduce engineering time.
- Offer hands‑on guidance and best‑practice patterns for deploying LLMs at scale.
- Enable clients to bring AI products to market faster and with higher quality.

---

## Bottom Line

Aurelio AI empowers AI engineers by delivering powerful open‑source tools, a comprehensive platform, and expert services—all centered around language models, agent design, and semantic information retrieval.

In [29]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [31]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

display(Markdown(few_shot_prompt.format()))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.

